# COVID 19 Reported Cases and Deaths
## A Local Perspective
All data automatically imported from John Hopkins University at https://github.com/CSSEGISandData/COVID-19

In [1]:
from __future__ import print_function
#from ipywidgets import interact, interactive, fixed, interact_manual, Dropdown
#import ipywidgets as wd
import pandas as pd
import numpy as np
#import plotly.graph_objects as go

In [2]:
#data pulled from John Hopkins University at https://github.com/CSSEGISandData/COVID-19
us_confirmed_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
us_death_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")

In [3]:
#cleaning data and preparing a single sheet for CA
us_confirmed_df.columns = map(str.lower, us_confirmed_df.columns)
us_death_df.columns = map(str.lower, us_death_df.columns)
today = us_death_df.columns[-1]
us_confirmed_df = us_confirmed_df.rename(columns = { 'province_state': 'state', 'admin2': 'county'})
us_death_df = us_death_df.rename(columns = {'province_state':'state', 'admin2':'county'})

# See the most affected counties in California

In [17]:
#@interact
state = 'California'
    #now need to sort this somehow
state_conf = us_confirmed_df[us_confirmed_df['state']==state]
state_dead = us_death_df[us_death_df['state']==state]
state_tot_lrg = pd.merge(state_dead.rename(columns={today:'dead'}), state_conf.rename(columns={today:'confirmed'}), on ='county')
state_tot_sml = state_tot_lrg[['county', 'confirmed', 'dead']]
    #might call graphing functions from here too
    #TODO: want to drop "Out of (statename)" counties and 'Unassigned'
state_tot_sml.sort_values(by = ['confirmed'], ascending = False).head(10)


,county,confirmed,dead
18,Los Angeles,195805,4760
32,Riverside,38487,738
29,Orange,38066,653
35,San Bernardino,34017,477
36,San Diego,30516,568
14,Kern,21228,151
9,Fresno,15759,138
38,San Joaquin,11958,180
0,Alameda,11909,192
42,Santa Clara,11030,192


## The Situation in Santa Barbara

In [20]:
state_tot_lrg = state_tot_lrg[state_tot_lrg['county']=='Santa Barbara'] 
state_tot_lrg = state_tot_lrg[['county', 'confirmed', 'dead']]
state_tot_lrg

,county,confirmed,dead
41,Santa Barbara,6526,63
